# Install, Paths and Parameters

In [1]:
# This extension reloads external Python files
import os
from pathlib import Path
import getpass
import numpy as np
import pandas as pd
import time
import math
from collections import defaultdict

import torch
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm
import random
import sys
from torch.utils.data import random_split
from matplotlib import pyplot as plt

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino
from src.model.train import *
from src.model.data import *

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')
# Path for intermediate outputs
BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc-fixed-labels/')
#BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc-subset/')

# Original Dataset
ORI_PATH = Path(DATA_PATH, 'ori_data/')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')

TR_PATH = Path(ORI_PATH, 'train/')
TR_ORI_LABEL_PATH = Path(TR_PATH,'correct_labels.txt')
TR_ORI_IMAGES_PATH = Path(TR_PATH,'images')

VAL_PATH = Path(ORI_PATH, 'validation/')
VAL_ORI_LABEL_PATH = Path(VAL_PATH,'correct_labels.txt')
VAL_ORI_IMAGES_PATH = Path(VAL_PATH,'images')

# DAmageNet
DN_PATH = Path(DATA_PATH, 'damageNet')
DN_LABEL_PATH = Path(DN_PATH, 'val_damagenet.txt')
DN_IMAGES_PATH = Path(DN_PATH, 'images')
DN_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'damagenet')
DN_POSTHOC_LABEL_PATH = Path(DN_POSTHOC_PATH, 'labels.csv')

# PGD
TR_PGD_PATH = Path(MAX_PATH, 'adversarial_data/pgd_06/train')
TR_PGD_LABEL_PATH = TR_ORI_LABEL_PATH
TR_PGD_IMAGES_PATH = Path(TR_PGD_PATH, 'images')
TR_PGD_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'pgd/train/')
TR_PGD_POSTHOC_LABEL_PATH = Path(TR_PGD_POSTHOC_PATH, 'labels.csv')

VAL_PGD_PATH = Path(MAX_PATH, 'adversarial_data/pgd_06/validation')
VAL_PGD_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_PGD_IMAGES_PATH = Path(VAL_PGD_PATH, 'images')
VAL_PGD_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'pgd/validation/')
VAL_PGD_POSTHOC_LABEL_PATH = Path(VAL_PGD_POSTHOC_PATH, 'labels.csv')

# CW
TR_CW_PATH = Path(MAX_PATH, 'adversarial_data/cw/train')
TR_CW_LABEL_PATH = TR_ORI_LABEL_PATH
TR_CW_IMAGES_PATH = Path(TR_CW_PATH, 'images')
TR_CW_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'cw/train/')
TR_CW_POSTHOC_LABEL_PATH = Path(TR_CW_POSTHOC_PATH, 'labels.csv')

VAL_CW_PATH = Path(MAX_PATH, 'adversarial_data/cw/validation')
VAL_CW_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_CW_IMAGES_PATH = Path(VAL_CW_PATH, 'images')
VAL_CW_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'cw/validation/')
VAL_CW_POSTHOC_LABEL_PATH = Path(VAL_CW_POSTHOC_PATH, 'labels.csv')

# FGSM
TR_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/train')
TR_FGSM_LABEL_PATH = TR_ORI_LABEL_PATH
TR_FGSM_IMAGES_PATH = Path(TR_FGSM_PATH, 'images')
TR_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/train/')
TR_FGSM_POSTHOC_LABEL_PATH = Path(TR_FGSM_POSTHOC_PATH, 'labels.csv')

VAL_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/validation')
VAL_FGSM_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_FGSM_IMAGES_PATH = Path(VAL_FGSM_PATH, 'images')
VAL_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/validation/')
VAL_FGSM_POSTHOC_LABEL_PATH = Path(VAL_FGSM_POSTHOC_PATH, 'labels.csv')

In [2]:
# If CLASS_SUBSET is specified, INDEX_SUBSET will be ignored. Set CLASS_SUBSET=None if you want to use indexes.
# INDEX_SUBSET = get_random_indexes(number_of_images = 50000, n_samples=1000)
# CLASS_SUBSET = get_random_classes(number_of_classes = 25, min_rand_class = 1, max_rand_class = 1001)


CLASS_SUBSET = np.load(CLASS_SUBSET_PATH)
# CLASS_SUBSET = CLASS_SUBSET[:2]

INDEX_SUBSET = None
NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 64

DEVICE = 'cuda'

In [3]:
datasets_paths = {
            'cw':{ 
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_CW_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_CW_IMAGES_PATH
                    }
                }
            },
            'ori':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_ORI_IMAGES_PATH
                    },
                    'val':{
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_ORI_IMAGES_PATH
                    }
                }
            },
            'dn':{
                'b':{
                    'train':{
                        'label':TR_CW_PATH,
                        'images':None
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':DN_IMAGES_PATH
                    }
                 }
            },
            'fgsm_06':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_FGSM_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_FGSM_IMAGES_PATH
                    }
                 }
            },
            'pgd_06':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_PGD_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_PGD_IMAGES_PATH
                    }
                }
            }
}

datasets = ['ori', 'cw', 'pgd_06', 'fgsm_06']
for ds in datasets:
    ds_dict = datasets_paths[ds]
    ds_dict['p'] = {
        'train': { 
            'images': Path(BASE_POSTHOC_PATH, ds, 'train', 'images'),
            'label': Path(BASE_POSTHOC_PATH, ds, 'train', 'labels.csv')
        },
        'val': { 
            'images': Path(BASE_POSTHOC_PATH, ds, 'val', 'images'),
            'label': Path(BASE_POSTHOC_PATH, ds, 'val', 'labels.csv')
        }
    }
    

In [4]:
datasets_paths['cw']

{'b': {'train': {'label': PosixPath('/cluster/scratch/thobauma/dl_data/ori_data/train/correct_labels.txt'),
   'images': PosixPath('/cluster/scratch/mmathys/dl_data/adversarial_data/cw/train/images')},
  'val': {'label': PosixPath('/cluster/scratch/thobauma/dl_data/ori_data/validation/correct_labels.txt'),
   'images': PosixPath('/cluster/scratch/mmathys/dl_data/adversarial_data/cw/validation/images')}},
 'p': {'train': {'images': PosixPath('/cluster/scratch/mmathys/dl_data/posthoc-fixed-labels/cw/train/images'),
   'label': PosixPath('/cluster/scratch/mmathys/dl_data/posthoc-fixed-labels/cw/train/labels.csv')},
  'val': {'images': PosixPath('/cluster/scratch/mmathys/dl_data/posthoc-fixed-labels/cw/val/images'),
   'label': PosixPath('/cluster/scratch/mmathys/dl_data/posthoc-fixed-labels/cw/val/labels.csv')}}}

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [5]:
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels)
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)
    
model, dino_classifier = get_dino()

linear_classifier = LinearClassifier(dino_classifier.linear.in_features, 
                         num_labels=len(CLASS_SUBSET))

linear_classifier.load_state_dict(torch.load("/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers/25_classes" + "/" + "clean.pt"))
linear_classifier.cuda()


from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit([i for i in CLASS_SUBSET])

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights.


LabelEncoder()

# Forward Pass

In [6]:
def posthoc_forward_pass(model, classifier, datasets, datasets_paths):
    logger_dict = {}
    for ds in datasets:
        ds_b = datasets_paths[ds]['b']
        ds_p = datasets_paths[ds]['p']
        
        logger_dict[ds] = {}
        
        transform = ONLY_NORMALIZE_TRANSFORM
        
        if ds == 'ori':
            transform = ORIGINAL_TRANSFORM
            
        for tv in ['train', 'val']:            
            print(f'''images: {ds_b[tv]['images']}\nlabel: {ds_b[tv]['label']}\npred: {ds_p[tv]['label']}''')
            
            data_set = AdvTrainingImageDataset(img_folder=ds_b[tv]['images'], 
                                               labels_file_name=ds_b[tv]['label'], 
                                               #labels_file_name='/cluster/home/thobauma/deeplearning/data-mmathys/adversarial_data_test/cw/train/labels.txt',
                                               transform=transform, 
                                               class_subset=CLASS_SUBSET,
                                               index_subset=None,
                                               label_encoder=label_encoder)
            
            data_loader = DataLoader(data_set, 
                                     batch_size=BATCH_SIZE, 
                                     num_workers=NUM_WORKERS, 
                                     pin_memory=PIN_MEMORY, 
                                     shuffle=False)
            
            print(f'''{ds}: {tv} {len(data_set)}''')
            logger_dict[ds][tv] = validate_network(model=model, 
                                                   classifier=classifier, 
                                                   validation_loader=data_loader, 
                                                   criterion=nn.CrossEntropyLoss(), 
                                                   tensor_dir=ds_p[tv]['images'],
                                                   adversarial_attack=None, 
                                                   n=4, 
                                                   avgpool_patchtokens=False, 
                                                   path_predictions=ds_p[tv]['label'],
                                                   show_image=False)
            
    return logger_dict

In [7]:
data_sets = ['ori', 'pgd_06', 'fgsm_06', 'cw']
logger_dict = posthoc_forward_pass(model,
                                   linear_classifier, 
                                   data_sets, 
                                   datasets_paths)

images: /cluster/scratch/mmathys/dl_data/adversarial_data/cw/train/images
label: /cluster/scratch/thobauma/dl_data/ori_data/train/correct_labels.txt
pred: /cluster/scratch/mmathys/dl_data/posthoc-fixed-labels/cw/train/labels.csv
cw: train 32181
saving predictions to: /cluster/scratch/mmathys/dl_data/posthoc-fixed-labels/cw/train/labels.csv
Test:  [  0/503]  eta: 0:06:13  loss: 0.103041 (0.103041)  acc1: 96.875000 (96.875000)  acc5: 100.000000 (100.000000)  time: 0.742024  data: 0.432813  max mem: 497
Test:  [ 20/503]  eta: 0:05:28  loss: 0.165014 (0.169806)  acc1: 93.750000 (94.717262)  acc5: 100.000000 (99.925595)  time: 0.677489  data: 0.405224  max mem: 498
Test:  [ 40/503]  eta: 0:05:13  loss: 0.148608 (0.164439)  acc1: 95.312500 (94.931402)  acc5: 100.000000 (99.923780)  time: 0.673042  data: 0.400707  max mem: 498
Test:  [ 60/503]  eta: 0:05:00  loss: 0.149335 (0.161389)  acc1: 95.312500 (95.210041)  acc5: 100.000000 (99.923156)  time: 0.680704  data: 0.409423  max mem: 498
Test:

# Create Labels

In [11]:
from functools import reduce

def merge_frames(frames, on_what=['file', 'true_labels'], how='left'):
    merged_df = reduce(lambda left, right:pd.merge(left, right, on=on_what, how=how,  suffixes=('', '_drop')), frames)
    merged_df.drop(merged_df.filter(regex='_drop$').columns.tolist(),axis=1, inplace=True)
    return merged_df

def get_merged_labels(datasets=['ori', 'cw', 'pgd_06', 'fgsm_06'], datasets_types=['train', 'val'], datasets_paths=datasets_paths, save_path=BASE_POSTHOC_PATH, get_df_dict = False):
    df_data_types = {}
    df_data = {}
    for tv in datasets_types:
        df_data[tv] = {}
        for ds in datasets:
            ds_dict = datasets_paths[ds]
            df_data[tv][ds] = pd.read_csv(ds_dict['p'][tv]['label'])
            df_data[tv][ds].rename(columns = {'pred_labels': ds+'_pred'}, inplace = True)
            if ds != 'ori':
                df_data[tv][ds] = pd.merge(df_data[tv][ds], df_data[tv]['ori'], on=['file', 'true_labels'], how='left')
            df_data[tv][ds].to_csv(Path(BASE_POSTHOC_PATH, ds, tv, 'labels_merged.csv'), sep=",", index=None)
        df_data_types[tv] = merge_frames(df_data[tv].values())
        if save_path is not None:
            df_data_types[tv].to_csv(Path(save_path,tv+'.csv'), sep=",", index=None)
    if get_df_dict:
        return df_data_types, df_data

    return df_data_types

    

In [12]:
df_types, df_data = get_merged_labels(get_df_dict=True)

In [13]:
df_types['train']

,file,true_labels,ori_pred,cw_pred,pgd_06_pred,fgsm_06_pred
0,n01873310_42267.JPEG,4,4,4,12,18
1,n02326432_39907.JPEG,11,11,11,13,9
2,n01608432_6128.JPEG,0,0,0,7,0
3,n04447861_3923.JPEG,23,23,23,18,18
4,n02101388_21983.JPEG,8,8,8,23,23
...,...,...,...,...,...,...
32176,n03777754_11639.JPEG,19,19,19,20,20
32177,n02927161_43568.JPEG,16,16,16,18,18
32178,n02398521_22023.JPEG,12,12,12,13,4
32179,n03888257_34756.JPEG,21,21,21,18,18


In [14]:
df_data['train']['cw']

,file,true_labels,cw_pred,ori_pred
0,n01873310_42267.JPEG,4,4,4
1,n02326432_39907.JPEG,11,11,11
2,n01608432_6128.JPEG,0,0,0
3,n04447861_3923.JPEG,23,23,23
4,n02101388_21983.JPEG,8,8,8
...,...,...,...,...
32176,n03777754_11639.JPEG,19,19,19
32177,n02927161_43568.JPEG,16,16,16
32178,n02398521_22023.JPEG,12,12,12
32179,n03888257_34756.JPEG,21,21,21


In [31]:
name = 'cw'
df = df_data['train'][name]

In [15]:
for name in ['cw', 'pgd_06', 'fgsm_06']:    
    df = df_data['train'][name]
    print(f'''{name}''')
    ldf = len(df)
    print(f'''total data:             {ldf}''')
    print(f'''correct pred:           {len(df[df['true_labels']==df['ori_pred']])},   {len(df[df['true_labels']==df['ori_pred']])/ldf}''')
    print(f'''incorrect adv pred:     {len(df[df['true_labels']!=df[name+'_pred']])},   {len(df[df['true_labels']!=df[name+'_pred']])/ldf}''')
    df_f = df[df['true_labels']==df['ori_pred']]
    print(f'''number adv tuples:      {len(df_f[df_f['true_labels']!=df_f[name+'_pred']])},   {len(df_f[df_f['true_labels']!=df_f[name+'_pred']])/ldf}''')
    print(f'''\n''')

cw
total data:             32181
correct pred:           31848,   0.9896522792952364
incorrect adv pred:     1528,   0.04748143314378049
number adv tuples:      1261,   0.039184612038159164


pgd_06
total data:             32181
correct pred:           31848,   0.9896522792952364
incorrect adv pred:     32181,   1.0
number adv tuples:      31848,   0.9896522792952364


fgsm_06
total data:             32181
correct pred:           31848,   0.9896522792952364
incorrect adv pred:     28047,   0.8715391069264473
number adv tuples:      27715,   0.8612224604580342




In [31]:
cw_df = pd.read_csv('/cluster/home/thobauma/deeplearning/data-mmathys/adversarial_data_test/cw/train/labels.txt', sep=" ", header=None)
cw_df.columns=['file', 'label']

In [19]:
cw_df

,file,cw_pred
0,n01873310_42267.JPEG,4
1,n02326432_39907.JPEG,11
2,n01608432_6128.JPEG,0
3,n04447861_3923.JPEG,23
4,n02101388_21983.JPEG,8
5,n01860187_2690.JPEG,3
6,n02101388_1842.JPEG,8
7,n01860187_18665.JPEG,3
8,n02101388_9950.JPEG,8
9,n02114712_1467.JPEG,9


In [29]:
cw_stuff = pd.merge(cw_df, df_data['train']['cw'], how='left', suffixes=('', '_post'), on=['file'])

In [30]:
cw_stuff

,file,cw_pred,true_labels,cw_pred_post,ori_pred
0,n01873310_42267.JPEG,4,4,4,4
1,n02326432_39907.JPEG,11,11,11,11
2,n01608432_6128.JPEG,0,0,0,0
3,n04447861_3923.JPEG,23,23,23,23
4,n02101388_21983.JPEG,8,8,8,8
5,n01860187_2690.JPEG,3,3,3,3
6,n02101388_1842.JPEG,8,8,8,8
7,n01860187_18665.JPEG,3,3,3,3
8,n02101388_9950.JPEG,8,8,8,8
9,n02114712_1467.JPEG,9,9,9,9
